In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Flatten
from tensorflow.keras.preprocessing import sequence
#from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SpatialDropout1D
#from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
MAX_WORDS = 100
MAX_LEN = 100
BATCH_SIZE=64
EPOCHS=100

In [3]:
df=pd.read_csv("/Users/abhinavshinow/Documents/GitHub/Mal_URL/Data/mal_1.csv")

In [4]:
df['type']=df['type'].replace({'phishing':0,'benign':1,'defacement':2,'malware':3})

In [5]:
df.head()

,url,type
0,br-icloud.com.br,0
1,mp3raid.com/music/krizz_kaliko.html,1
2,bopsecrets.org/rexroth/cr/1.htm,1
3,http://www.garage-pirenne.be/index.php?option=...,2
4,http://adventure-nicaragua.net/index.php?optio...,2


In [6]:
x=df['url']
y=df['type']

#Tokenising the url's
tok = Tokenizer(num_words=MAX_WORDS)
tok.fit_on_texts(x)
x = tok.texts_to_sequences(x.values)
x = sequence.pad_sequences(x, maxlen=MAX_LEN)

#One-Hot Encoding the target labels
y = np.array(to_categorical(y))

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,shuffle=True)

In [7]:
model = Sequential()
# model.add(Embedding(MAX_WORDS, 300, input_length=MAX_LEN))
# model.add(SpatialDropout1D(0.2))
# model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.1,return_sequences=True,activation='relu'))
# model.add(Dense(32,activation='relu'))
# model.add(Dense(4,activation='softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print(model.summary())
model = Sequential()
model.add(Embedding(MAX_WORDS, 300, input_length=MAX_LEN))
model.add(SpatialDropout1D(0.1))
model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.1,return_sequences=True, activation='relu'))
model.add(Flatten())
model.add(Dense(4, name='out_layer', activation="softmax"))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2021-11-26 21:27:05.525735: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-26 21:27:05.526275: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          30000     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 100, 300)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100, 32)           42624     
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
out_layer (Dense)            (None, 4)                 12804     
Total params: 85,428
Trainable params: 85,428
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
es=EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=3,
        restore_best_weights=True, 
        verbose=1,
    )
checkpoint_cb=ModelCheckpoint(
        "embedding_ulr.h5",
        save_best_only=True,
        monitor='val_loss',
        mode='min',
    )
reduce_lr=ReduceLROnPlateau(
        monitor='val_loss',
        patience=6,
        mode='min',
        verbose=1,
    )
callbacks=[es,checkpoint_cb,reduce_lr]

In [9]:
# steps_per_epoch = int( np.ceil(x_train.shape[0] / BATCH_SIZE) )

In [10]:
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs = EPOCHS,steps_per_epoch=128,validation_data=(x_test,y_test),callbacks=callbacks)

Epoch 1/100


2021-11-26 21:27:05.775943: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-26 21:27:05.778941: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-11-26 21:27:06.553050: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


  5/128 [>.............................] - ETA: 46:51 - loss: 1.2531 - accuracy: 0.5500